In [ ]:
import pandas as pd 
import numpy as np
import math
from datetime import datetime

In [ ]:
data = pd.read_csv("realEmtHashed.csv")

In [ ]:
data.head()

# Recatagorizing Data for Sender Accounts
## Changed the column headings to the sender account number with attributes about the sender account as the rows.

In [4]:
test_len = 10000
data_len = len(data)
new_data = {}
start_time = datetime.now()

for i in range(test_len): 
    row = data.iloc[i]
    sender = row['userID']
    amount = row['money_amount']
    location = (row['latitude'],row['longitude'])
    time = row['timestamp']
    ISP = row['isp']
    receiver = row['payeeEmail']
    
    if sender not in new_data: 
        new_data[sender] = {'transactions':1,'amounts':[],'locations':[],'receivers':{},'time_stamps':[],'senderISP':[]}
    else:
        new_data[sender]['transactions'] += 1
    
    new_data[sender]['amounts'].append(amount)
    new_data[sender]['time_stamps'].append(time)

    if location not in new_data[sender]['locations']:
        new_data[sender]['locations'].append(location)
        
    if ISP not in new_data[sender]['senderISP']:
        new_data[sender]['senderISP'].append(ISP)
        
    if receiver in new_data[sender]['receivers']:
        new_data[sender]['receivers'][receiver] += 1 
    else: 
        new_data[sender]['receivers'][receiver] = 1
        
duration = datetime.now() - start_time
print("Parsed data in: ", duration)

Parsed data in:  0:00:02.591485


# Saving New Sender Dataset

In [5]:
sender = pd.DataFrame.from_dict(new_data)

In [6]:
sender.head()

,044f9d391a27b59859fb3b274237671ff246bb9b69ae7d66db3a7a3875b9520d,3ec92838d15518ea50355e7adfa01d470a7e49479c426db3ffc65befe6a2e622,0f772a1e33aec1998c7a917e5cf67f30eeb485db693c57da470cf67d340c2453,30e1bb678ca1426e122bdb953a2320abbacd14a1dd093b77a4b2f33d84fc937e,5f1331872f036401287146d98e5b7f9d23a24a789b4d72c3731dcabfc67bbc48,a844f08602b27f68ecbe4685991c427948dc3ade39775ea3d29e99bc1c789c1d,800012ea1c431863b68eff6dbf714090466718395a6158b76e62b9e16b63d6f8,01fad62c3e5b6cbf95a26d59abd3249796e56a1caa027b013b470e06ffc4afba,8b11e3e97273699aeca08dbb84f884921c53f7af83144143166db368cb1b7df7,831b7e058c9542cea19d2fecbebf0e838ee5a0ccaabe06e9b13eab2940cd9c0f,...,e5c5ac9bf2d3b1b114043affb014ad06b1e50b9260ed033151afb18c26f499f6,8b566b63fef7abd0f0dcc8922843e912785d874fe04d1b24940041e783352539,4d42f17b6c4d47cf80eae6a320b95a085b432072872d231bb622e6f95648a58d,756c9da3bf360f62768fb6b7e773ed34225258114e6d098cbdf66d62011e63e9,7ac76c06631a5afc4653c863e5b54bf6c8c1c373c1cefc7c621878ced76d727e,be48cb3dd8e3c86de691d529f629389d644c58edd95ff2c0e3a303885eaf869f,5a235665dc8c09a3eb1de698157d411f58c0ececc41416959a421441e639f1fe,9b77150c5a7ef68ca2f11936739540e99ccb3ce0c21fe594f36ea48091c4c59b,6af6f566da1e80d797352fc9f17e1cff9218d0e8b49932020da1a85424f54822,c7be6a75a5cfb169e57422f4833933b3e8c4399e2ace222b86ce03f80046899e
transactions,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
amounts,[500.0],[2571.0],[1025.0],[1510.0],[69.0],[820.0],[666.66],[350.0],[2776.57],[200.0],...,[207.9],[480.0],[845.0],[300.0],[925.0],[1050.0],[1550.0],[3000.0],[179.0],[50.0]
locations,"[(44.53, -78.75)]","[(43.89, -78.88)]","[(45.73, -73.71)]","[(43.65, -79.42)]","[(45.55, -73.58)]","[(50.05, -119.29)]","[(48.48, -123.37)]","[(45.33, -75.67)]","[(49.28, -123.11)]","[(49.92, -96.96)]",...,"[(53.91, -122.75)]","[(51.08, -113.96)]","[(45.44, -73.46)]","[(49.93, -105.97)]","[(49.26, -123.16)]","[(43.87, -79.44)]","[(49.19, -122.94)]","[(51.03, -113.82)]","[(43.65, -79.48)]","[(43.78, -79.42)]"
receivers,{'2fe5ce59f8fbac0fccaca974b9bb08ab4b2afe3d5703...,{'480e4ce89838a108880a7ba13475d9aa92e995476a86...,{'02fc4c55153e0f1645e8c3b24a0d225aa09a3e6806a6...,{'7e28bb7504342ed3806218195ab31d7dcab330cc7511...,{'42ad4701060ceaebd11b09360ca9a24d75d217f90921...,{'a81fd73419341dda2674163e8c5916943968d2857aa8...,{'922efec678a4b9ef3feb83cfa94c2cb3328864a3ea4c...,{'bf6f30e2e6e31010cfcd15eb765b7a4cfc21ff33fe29...,{'99802f30b2029bcc495725c61b51464a4bd5a85ccbf3...,{'f2717c125ac73191416ed910bee13fbaaeb1c90d8a7b...,...,{'ff19617e9adbab696de747eec0be5c092929ebcdff25...,{'291c9c7d6e88da402d21bba58d9e0ed2db0d542f1b11...,{'d667389426be58e330cd61b301a71f4e85e9bec57ede...,{'798c4f48f2a41b51f3b790035465db3c790c82b3dbac...,{'5a5a7451fa52fb233c268b9d02000770659f52a5aa67...,{'5823f965be7adcaaffaf898245f879229a1f38b316b6...,{'b510dffd23ec87b55012a779ab2be3de8086ce2e653f...,{'cc3778e3b198541c67478b0332fa8a77112d0497c0a0...,{'8772b440ab2fa81ce2a27a0a0f3f71d342d016d9de78...,{'854e7667a9b0a2b0cee86d0eb2f4bea01dd1a35a7c51...
time_stamps,[2018-06-01 19:02:54.026-0400],[2018-06-01 05:37:11.480-0400],[2018-06-01 00:44:01.691-0400],[2018-06-01 00:33:22.304-0400],[2018-06-01 19:27:57.245-0400],[2018-06-01 00:12:43.788-0400],[2018-06-01 11:05:43.082-0400],[2018-06-01 14:00:51.380-0400],[2018-06-01 15:52:35.582-0400],[2018-06-01 08:22:02.319-0400],...,[2018-06-01 16:05:34.136-0400],[2018-06-01 18:26:56.105-0400],[2018-06-01 06:35:16.467-0400],[2018-06-01 12:46:47.714-0400],[2018-06-01 13:34:44.598-0400],[2018-06-01 09:22:47.131-0400],[2018-06-01 10:41:38.070-0400],[2018-06-01 08:50:27.956-0400],[2018-06-01 14:40:53.559-0400],[2018-06-01 07:54:21.376-0400]


In [7]:
def unique_senders(dataset): 
    different_accounts = [] 
    for name in dataset.iteritems(): 
        value = name[1]['receivers']
        different_accounts.append(len(value.keys()))
    return different_accounts

In [8]:
def unique_receivers(dataset): 
    different_accounts = [] 
    for name in dataset.iteritems(): 
        value = name[1]['senders']
        different_accounts.append(len(value.keys()))
    return different_accounts

In [9]:
def mean_amount(dataset):
    mean_amount = []   

    #Mean transaction amount per account ID
    for name in dataset.iteritems():
        value = name[1]['amounts']
        amount = 0 
        bad_values = 0
        #Using a loop and not sum because the some of the fucks in here are strings
        for item in value:
            try:
                amount += float(item)
            except ValueError: 
                bad_values += 1 

        transaction_number = len(value) - bad_values

        if transaction_number != 0:
            mean = amount/transaction_number
            mean_amount.append(mean)
        else: 
            mean_amount.append(0)
    return mean_amount

In [10]:
def stnd_dev(dataset):
    #Standard Deviation for amounts 
    std_dev = []
    z_score = [] 
    for name in dataset.iteritems(): 
        value = name[1]['amounts']
        mean = name[1]['mean_transaction_amount']
        variance = 0 
        bad_values = 0 
        for item in value: 
            try: 
                variance += math.pow((float(item) - mean),2)
            except: 
                bad_values += 1

        transaction_number = len(value) - bad_values 
        if transaction_number != 0: 
            std = math.sqrt(variance)/transaction_number
            std_dev.append(std)
        else: 
            std_dev.append(0)
    return std_dev

In [11]:
def ISP_changes(dataset):
    #Number of Different ISP's 
    ISP_changes = [] 

    for name in dataset.iteritems(): 
        value = name[1]['senderISP']
        ISP_changes.append(len(value))
    return ISP_changes

In [12]:
def timeBetweenFirstLast(dates):
    '''
    Given a list of dates of the form %Y-%m-%d %H:%M:%S.%f%z return a datetime.timedelta
    object that contains the amount of time between the first and last (chronologically) dates
    '''
    # convert to datetime objects
    datetimes = []
    for date in dates:
        datetimes.append(datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f%z"))
     
    # sort chronologically
    datetimes.sort()

    # return time between first and last dates
    return np.abs(datetimes[-1] - datetimes[0])

def trans_freq(dataset):
    transaction_frequency = [] 
    for name in dataset.iteritems(): 
        value = name[1]['time_stamps']
        difference = timeBetweenFirstLast(value)
        transaction_frequency.append(((difference/name[1]['transactions']).total_seconds())/86400)
    return transaction_frequency

In [13]:
def distance_from_us(dataset):
    #Distance from the American Border
    distance = []

    for name in dataset.iteritems(): 
        value = name[1]['locations'][0]
        y,x = value

        if x >= -125.680 and x <= -95.190 or x < -125.680: 
            border = 49.00 

        elif x > -95.190 and x <= -82.840: 
            border = -0.591*x - 7.267

        elif x > -82.840 and x <= -68.912: 
            border = 0.402*x + 75

        elif x > -68.912 and x <= -66.890: 
            border = -2.850*x - 149.159

        elif x > -95.19 and x <= -82.84: 
            border = -0.591*x - 7.267

        else: 
            border = 44.7 

        distance.append(abs(y - border))
        
    dataset.loc['distance_from_border'] = distance
    # Replacing all the nan values with the average
    count = 0
    total = 0 

    for name in dataset.iteritems():
        value = name[1]['distance_from_border']
        if not np.isnan(value):
            count += 1
            total += value

    average = total/count

    for i in range(len(distance)): 
        if np.isnan(distance[i]): 
            distance[i] = average

    return distance

# Features For Sender Dataset

In [14]:
sender.loc['mean_transaction_amount'] = mean_amount(sender)
sender.loc['transacton_std_dev'] = stnd_dev(sender)
sender.loc['total_ISPs'] = ISP_changes(sender)
sender.loc['transaction_frequency'] = trans_freq(sender)
sender.loc['distance_from_border'] = distance_from_us(sender)
sender.loc['unique_receivers'] = unique_senders(sender)
#Total Transactions is also a feature included

In [15]:
sender.head(15)

,044f9d391a27b59859fb3b274237671ff246bb9b69ae7d66db3a7a3875b9520d,3ec92838d15518ea50355e7adfa01d470a7e49479c426db3ffc65befe6a2e622,0f772a1e33aec1998c7a917e5cf67f30eeb485db693c57da470cf67d340c2453,30e1bb678ca1426e122bdb953a2320abbacd14a1dd093b77a4b2f33d84fc937e,5f1331872f036401287146d98e5b7f9d23a24a789b4d72c3731dcabfc67bbc48,a844f08602b27f68ecbe4685991c427948dc3ade39775ea3d29e99bc1c789c1d,800012ea1c431863b68eff6dbf714090466718395a6158b76e62b9e16b63d6f8,01fad62c3e5b6cbf95a26d59abd3249796e56a1caa027b013b470e06ffc4afba,8b11e3e97273699aeca08dbb84f884921c53f7af83144143166db368cb1b7df7,831b7e058c9542cea19d2fecbebf0e838ee5a0ccaabe06e9b13eab2940cd9c0f,...,e5c5ac9bf2d3b1b114043affb014ad06b1e50b9260ed033151afb18c26f499f6,8b566b63fef7abd0f0dcc8922843e912785d874fe04d1b24940041e783352539,4d42f17b6c4d47cf80eae6a320b95a085b432072872d231bb622e6f95648a58d,756c9da3bf360f62768fb6b7e773ed34225258114e6d098cbdf66d62011e63e9,7ac76c06631a5afc4653c863e5b54bf6c8c1c373c1cefc7c621878ced76d727e,be48cb3dd8e3c86de691d529f629389d644c58edd95ff2c0e3a303885eaf869f,5a235665dc8c09a3eb1de698157d411f58c0ececc41416959a421441e639f1fe,9b77150c5a7ef68ca2f11936739540e99ccb3ce0c21fe594f36ea48091c4c59b,6af6f566da1e80d797352fc9f17e1cff9218d0e8b49932020da1a85424f54822,c7be6a75a5cfb169e57422f4833933b3e8c4399e2ace222b86ce03f80046899e
transactions,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
amounts,[500.0],[2571.0],[1025.0],[1510.0],[69.0],[820.0],[666.66],[350.0],[2776.57],[200.0],...,[207.9],[480.0],[845.0],[300.0],[925.0],[1050.0],[1550.0],[3000.0],[179.0],[50.0]
locations,"[(44.53, -78.75)]","[(43.89, -78.88)]","[(45.73, -73.71)]","[(43.65, -79.42)]","[(45.55, -73.58)]","[(50.05, -119.29)]","[(48.48, -123.37)]","[(45.33, -75.67)]","[(49.28, -123.11)]","[(49.92, -96.96)]",...,"[(53.91, -122.75)]","[(51.08, -113.96)]","[(45.44, -73.46)]","[(49.93, -105.97)]","[(49.26, -123.16)]","[(43.87, -79.44)]","[(49.19, -122.94)]","[(51.03, -113.82)]","[(43.65, -79.48)]","[(43.78, -79.42)]"
receivers,{'2fe5ce59f8fbac0fccaca974b9bb08ab4b2afe3d5703...,{'480e4ce89838a108880a7ba13475d9aa92e995476a86...,{'02fc4c55153e0f1645e8c3b24a0d225aa09a3e6806a6...,{'7e28bb7504342ed3806218195ab31d7dcab330cc7511...,{'42ad4701060ceaebd11b09360ca9a24d75d217f90921...,{'a81fd73419341dda2674163e8c5916943968d2857aa8...,{'922efec678a4b9ef3feb83cfa94c2cb3328864a3ea4c...,{'bf6f30e2e6e31010cfcd15eb765b7a4cfc21ff33fe29...,{'99802f30b2029bcc495725c61b51464a4bd5a85ccbf3...,{'f2717c125ac73191416ed910bee13fbaaeb1c90d8a7b...,...,{'ff19617e9adbab696de747eec0be5c092929ebcdff25...,{'291c9c7d6e88da402d21bba58d9e0ed2db0d542f1b11...,{'d667389426be58e330cd61b301a71f4e85e9bec57ede...,{'798c4f48f2a41b51f3b790035465db3c790c82b3dbac...,{'5a5a7451fa52fb233c268b9d02000770659f52a5aa67...,{'5823f965be7adcaaffaf898245f879229a1f38b316b6...,{'b510dffd23ec87b55012a779ab2be3de8086ce2e653f...,{'cc3778e3b198541c67478b0332fa8a77112d0497c0a0...,{'8772b440ab2fa81ce2a27a0a0f3f71d342d016d9de78...,{'854e7667a9b0a2b0cee86d0eb2f4bea01dd1a35a7c51...
time_stamps,[2018-06-01 19:02:54.026-0400],[2018-06-01 05:37:11.480-0400],[2018-06-01 00:44:01.691-0400],[2018-06-01 00:33:22.304-0400],[2018-06-01 19:27:57.245-0400],[2018-06-01 00:12:43.788-0400],[2018-06-01 11:05:43.082-0400],[2018-06-01 14:00:51.380-0400],[2018-06-01 15:52:35.582-0400],[2018-06-01 08:22:02.319-0400],...,[2018-06-01 16:05:34.136-0400],[2018-06-01 18:26:56.105-0400],[2018-06-01 06:35:16.467-0400],[2018-06-01 12:46:47.714-0400],[2018-06-01 13:34:44.598-0400],[2018-06-01 09:22:47.131-0400],[2018-06-01 10:41:38.070-0400],[2018-06-01 08:50:27.956-0400],[2018-06-01 14:40:53.559-0400],[2018-06-01 07:54:21.376-0400]
senderISP,[Cable Cable],[Primus Telecommunications Canada],[Bell Canada],[Amanah Tech],[Distributel Communications],[Shaw Communications],[Shaw Communications],[Shared Services Canada],[Bell Canada],[Shaw Communications],...,[Shaw Communications],[CIK Telecom],[ACN DSL],[SaskTel],[Telus Communications],[York Region District School Board],[Telus Communications],[Telus Communications],

# Clustering Data

In [ ]:
import sklearn
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt

In [ ]:
cluster_amount = 5

In [ ]:
mstd = [] 
m_total_transactions = [] 
mtransf = [] 
mean_ISP = [] 
mean_border = [] 

transf_std = [] 
transf_total_transactions = [] 
transf_ISP = [] 
transf_border = [] 

ISP_std = [] 
ISP_total_transactions = [] 
ISP_border = [] 

total_transactions_std = [] 
total_transactions_border = [] 

border_std = [] 

for name in sender.iteritems(): 
    account_num = name[0]
    value = name[1]
    
    total_transaction = value['transactions']
    mean_trans = value['mean_transaction_amount']
    std_trans = value['transacton_std_dev']
    trans_frequency = value['transaction_frequency']
    senderISP = value['total_ISPs']
    distance_us = value['distance_from_border']
    
    mstd.append([mean_trans,std_trans])
    m_total_transactions.append([mean_trans,total_transaction])
    mtransf.append([trans_frequency,mean_trans])
    mean_ISP.append([senderISP,mean_trans])
    mean_border.append([distance_us,mean_trans])
    
    transf_std.append([trans_frequency,std_trans])
    transf_total_transactions.append([trans_frequency,total_transaction])
    transf_ISP.append([senderISP,trans_frequency])
    transf_border.append([distance_us,trans_frequency])
    
    ISP_total_transactions.append([senderISP,total_transaction])
    ISP_std.append([senderISP,std_trans])
    ISP_border.append([senderISP,distance_us])
    
    total_transactions_std.append([total_transaction,std_trans])
    total_transactions_border.append([total_transaction,distance_us]) 

    border_std.append([distance_us,std_trans])

    
all_features = [mstd, m_total_transactions, mtransf, mean_ISP, mean_border, transf_std, transf_total_transactions,
               transf_ISP, transf_border, ISP_std, ISP_total_transactions,ISP_border,total_transactions_std, 
               total_transactions_std, total_transactions_border, border_std] 

for i in range(len(all_features)): 
    all_features[i] = sklearn.preprocessing.scale(all_features[i])

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[0])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[0]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Mean Transaction Amount vs. Spontenaity in Transaction Amount')
plt.ylabel('Transaction Amount Standard Deviation')
plt.xlabel('Mean Transaction Amount')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[1])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[1]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Mean Transaction Amount vs. Total Transactions')
plt.ylabel('Total Transactions')
plt.xlabel('Mean Transaction Amount')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[2])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[2]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Average Transaction Period vs. Mean Transaction Amount')
plt.xlabel('Average Transaction Gap')
plt.ylabel('Mean Transaction Amount')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[3])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[3]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Total Number of ISPs Used vs. Mean Transaction Amount')
plt.xlabel('Total Number of ISPs Used')
plt.ylabel('Mean Transaction Amount')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[5])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[5]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Average Transaction Period vs. Transaction Amount Standard Deviation')
plt.ylabel('Transaction Amount Standard Deviation')
plt.xlabel('Average Transaction Period')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[6])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[6]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Average Transaction Period vs. Total Number of Transactions')
plt.ylabel('Total Number of Transactions')
plt.xlabel('Average Transaction Period')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[7])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[7]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Number of ISPs Used vs. Average Transaction Period')
plt.xlabel('Number of ISPs Used')
plt.ylabel('Average Transaction Period')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[9])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[9]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Number of ISPs Used vs. Transaction Amount Standard Deviation')
plt.ylabel('Transaction Amount Standard Deviation')
plt.xlabel('Number of ISPs Used')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[10])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[10]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Number of ISPs Used vs. Total Number of Transactons')
plt.ylabel('Total Number of Transactions')
plt.xlabel('Number of ISPs Used')
plt.show()

In [ ]:
#print(np.shape(cluster_data_mstd))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[12])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[12]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Total Number of Transactions vs. Transaction Amount Standard Deviation')
plt.ylabel('Transaction Amount Standard Deviation')
plt.xlabel('Total Number of Transactions')
plt.show()

In [ ]:
#print(np.shape(mean_border))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[4])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[4]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Distance from US border vs. Mean Transaction Amount')
plt.ylabel('Mean Transaction Amount')
plt.xlabel('Distance from US border')
plt.show()

In [ ]:
#print(np.shape(mean_border))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[8])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[8]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Distance from US border vs. Transaction Frequency')
plt.ylabel('Transaction Frequency')
plt.xlabel('Distance from US border')
plt.show()

In [ ]:
#print(np.shape(mean_border))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[11])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[11]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Total ISPs Used vs. Distance from US border')
plt.xlabel('Total ISPs Used')
plt.ylabel('Distance from US border')
plt.show()

In [ ]:
#print(np.shape(mean_border))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[14])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[14]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Total Transactions vs. Distance from US border')
plt.xlabel('Total Transactions')
plt.ylabel('Distance from US border')
plt.show()

In [ ]:
#print(np.shape(mean_border))
kmeans = KMeans(n_clusters=cluster_amount, random_state=0).fit(all_features[15])
centers = kmeans.cluster_centers_
labels = kmeans.labels_
np.shape(centers)

x = [] 
y = []

center_x = [] 
center_y = [] 

for point in all_features[15]: 
    x.append(point[0])
    y.append(point[1])
    
for point in centers: 
    center_x.append(point[0])
    center_y.append(point[1])
    
plt.scatter(x,y,c=labels.astype(float))
plt.scatter(center_x,center_y,marker="x",s=200)
plt.title('Distance from US border vs. Average Transaction Standard Deviation ')
plt.xlabel('Distance from US border')
plt.ylabel('Average Transaction Standard Deviation')
plt.show()

# Recatagorizing Data for Receiver Emails
## Changed the column headings to the sender account number with attributes about the Receiver Email as the rows.

In [16]:
test_len = 10000
data_len = len(data)
new_data = {}
start_time = datetime.now()

for i in range(test_len): 
    row = data.iloc[i]
    sender = row['userID']
    amount = row['money_amount']
    location = (row['latitude'],row['longitude'])
    time = row['timestamp']
    ISP = row['isp']
    receiver = row['payeeEmail']
    
    if receiver not in new_data: 
        new_data[receiver] = {'transactions':1,'amounts':[],'senders':{},'time_stamps':[]}
    else:
        new_data[receiver]['transactions'] += 1
    
    new_data[receiver]['amounts'].append(amount)
    new_data[receiver]['time_stamps'].append(time)
        
    if sender in new_data[receiver]['senders']:
        new_data[receiver]['senders'][sender] += 1 
    else: 
        new_data[receiver]['senders'][sender] = 1
        
duration = datetime.now() - start_time
print("Parsed data in: ", duration)

Parsed data in:  0:00:02.634899


In [17]:
receiver = pd.DataFrame.from_dict(new_data)

# Features For Receiver Dataset

In [18]:
receiver.loc['mean_transaction_amount'] = mean_amount(receiver)
receiver.loc['transacton_std_dev'] = stnd_dev(receiver)
receiver.loc['transaction_frequency'] = trans_freq(receiver)
receiver.loc['unique_senders'] = unique_receivers(receiver)
#Total Transactions also a feature which is included

In [19]:
receiver.head(15)

,2fe5ce59f8fbac0fccaca974b9bb08ab4b2afe3d5703d110a400db4053f82f74,480e4ce89838a108880a7ba13475d9aa92e995476a86ea093ec85ba9561bff19,02fc4c55153e0f1645e8c3b24a0d225aa09a3e6806a6523e023b951f8ccd424f,7e28bb7504342ed3806218195ab31d7dcab330cc7511ad7e99470d35faf27097,42ad4701060ceaebd11b09360ca9a24d75d217f90921deecae5fe2238750f76c,a81fd73419341dda2674163e8c5916943968d2857aa829450c1177858ebcec6c,922efec678a4b9ef3feb83cfa94c2cb3328864a3ea4c6fb4e2b252340eb8949d,bf6f30e2e6e31010cfcd15eb765b7a4cfc21ff33fe295b2629a1f7aa7560a348,99802f30b2029bcc495725c61b51464a4bd5a85ccbf3080468d4684f69b99ded,f2717c125ac73191416ed910bee13fbaaeb1c90d8a7b2d789aeeafc854fddb67,...,ff19617e9adbab696de747eec0be5c092929ebcdff256fff8210e3aba415bc52,291c9c7d6e88da402d21bba58d9e0ed2db0d542f1b11ea0f74d05ea72d1ba14f,d667389426be58e330cd61b301a71f4e85e9bec57edeae098b9c7b692c2b2d34,798c4f48f2a41b51f3b790035465db3c790c82b3dbac2e2d363e5e194d3a73fb,5a5a7451fa52fb233c268b9d02000770659f52a5aa677f4b90654b86f5b21519,5823f965be7adcaaffaf898245f879229a1f38b316b652b4528cd98a9f8411e1,b510dffd23ec87b55012a779ab2be3de8086ce2e653f0768e4003c671abcb752,cc3778e3b198541c67478b0332fa8a77112d0497c0a07915692f2426c703b7c8,8772b440ab2fa81ce2a27a0a0f3f71d342d016d9de78939f37cee8b37db471c8,854e7667a9b0a2b0cee86d0eb2f4bea01dd1a35a7c51025dd9c8bda41d861011
transactions,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
amounts,[500.0],[2571.0],[1025.0],[1510.0],[69.0],[820.0],[666.66],[350.0],[2776.57],[200.0],...,[207.9],[480.0],[845.0],[300.0],[925.0],[1050.0],[1550.0],[3000.0],[179.0],[50.0]
senders,{'044f9d391a27b59859fb3b274237671ff246bb9b69ae...,{'3ec92838d15518ea50355e7adfa01d470a7e49479c42...,{'0f772a1e33aec1998c7a917e5cf67f30eeb485db693c...,{'30e1bb678ca1426e122bdb953a2320abbacd14a1dd09...,{'5f1331872f036401287146d98e5b7f9d23a24a789b4d...,{'a844f08602b27f68ecbe4685991c427948dc3ade3977...,{'800012ea1c431863b68eff6dbf714090466718395a61...,{'01fad62c3e5b6cbf95a26d59abd3249796e56a1caa02...,{'8b11e3e97273699aeca08dbb84f884921c53f7af8314...,{'831b7e058c9542cea19d2fecbebf0e838ee5a0ccaabe...,...,{'e5c5ac9bf2d3b1b114043affb014ad06b1e50b9260ed...,{'8b566b63fef7abd0f0dcc8922843e912785d874fe04d...,{'4d42f17b6c4d47cf80eae6a320b95a085b432072872d...,{'756c9da3bf360f62768fb6b7e773ed34225258114e6d...,{'7ac76c06631a5afc4653c863e5b54bf6c8c1c373c1ce...,{'be48cb3dd8e3c86de691d529f629389d644c58edd95f...,{'5a235665dc8c09a3eb1de698157d411f58c0ececc414...,{'9b77150c5a7ef68ca2f11936739540e99ccb3ce0c21f...,{'6af6f566da1e80d797352fc9f17e1cff9218d0e8b499...,{'c7be6a75a5cfb169e57422f4833933b3e8c4399e2ace...
time_stamps,[2018-06-01 19:02:54.026-0400],[2018-06-01 05:37:11.480-0400],[2018-06-01 00:44:01.691-0400],[2018-06-01 00:33:22.304-0400],[2018-06-01 19:27:57.245-0400],[2018-06-01 00:12:43.788-0400],[2018-06-01 11:05:43.082-0400],[2018-06-01 14:00:51.380-0400],[2018-06-01 15:52:35.582-0400],[2018-06-01 08:22:02.319-0400],...,[2018-06-01 16:05:34.136-0400],[2018-06-01 18:26:56.105-0400],[2018-06-01 06:35:16.467-0400],[2018-06-01 12:46:47.714-0400],[2018-06-01 13:34:44.598-0400],[2018-06-01 09:22:47.131-0400],[2018-06-01 10:41:38.070-0400],[2018-06-01 08:50:27.956-0400],[2018-06-01 14:40:53.559-0400],[2018-06-01 07:54:21.376-0400]
mean_transaction_amount,500,2571,1025,1510,69,820,666.66,350,2776.57,200,...,207.9,480,845,300,925,1050,1550,3000,179,50
transacton_std_dev,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
transaction_frequency,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
unique_senders,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
